In [ ]:
# Cell 1: Imports
import pandas as pd

# Cell 2: Load Excel calendar
xlsx_path = r'C:\temp\timor_leste\aileu_crop_calendar.xlsx'
df = pd.read_excel(xlsx_path)

# Cell 3: Identify month columns
months = [
    'January','February','March','April','May','June',
    'July','August','September','October','November','December'
]
# Find which columns in df match our month list (case-insensitive)
month_cols = [col for col in df.columns if col.strip().capitalize() in months]

# Cell 4: Specify the ID & crop columns in your sheet
id_cols = ['adm2_pcode', 'crop']

# Cell 5: Melt to long format and drop blanks
long = (
    df
    .melt(
        id_vars=id_cols,
        value_vars=month_cols,
        var_name='month',
        value_name='phase'
    )
    .dropna(subset=['phase'])
)

# Cell 6: Convert month names to numbers
month_map = {m: i+1 for i, m in enumerate(months)}
long['month_num'] = long['month'].str.strip().str.capitalize().map(month_map)

# Cell 7: Aggregate to get start/end month per (polygon, crop, phase)
ranges = (
    long
    .groupby(id_cols + ['phase'], as_index=False)
    .agg(
        start_month=('month_num', 'min'),
        end_month=('month_num',   'max')
    )
)

# Cell 8: Save to CSV
out_csv = r'C:\temp\timor_leste\updated_crop_calendar.csv'
ranges.to_csv(out_csv, index=False)
print(f'Written tidy calendar to: {out_csv}')

# Cell 9: Preview
ranges.head(10)
